# Chapter 14: Classifying Images with Deep CNNs 

Chapter layout: 

1. Understanding Convolutions 
2. Implementing a CNN 
3. Implementing a Deep CNN with PyTorch 
4. Smile Classification using a CNN

### Understanding Convolutions

* CNNs are a family of models that were inspired from the visual cortext of the human brain. 
    * Neurons responds differently to light: the primary layer detects edges, higher order layers detect shapes and patterns
* Orginalally developed by Yann LeCun and colleagues in the 1990s
* CNNs are usually refered to as _feature extraction layers_ 


CNNs are usually thought of as feature extraction layers that are able to extract low-level features in the early layers which are utilized later in the network to detect patterns between these features and the target variable of interest. 

CNNs construct a *feature hierachy* by combining low-level features to form high-level features.

CNN computes *feature maps* from an input image to create a feature. They look over _local receptive fields_ = local patch of pixels to construct and pool features in small areas of the image. 

This method relies on 
* sparse connectivity = an element of a feature map is only connected to its nearest neighbor 
* parameter sharing = by learning a common set of parameters in a patch, we can utilize it in different parts of the image 



Typically a CNN is constructed of 

1. Several _convolutional_ layers 
2. Subsampling / Pooling layers
    * These don't have any weights or biases, just a useful aggregator
3. Fully connected layers



**A discrete convolution in 1D**

A convolution for two discrete vectors is $x$ and $y$ is given by 

$$ z = x * y \rightarrow z_i = \sum_{k=\infty}^{\infty}x[i-k]y[k]$$

where it is assumed $x[i] = y[j] = 0$ for all non-index elements of $x$ and $y$.

The process of filling these zeros is called _zero padding_. In addition to utilize the dot prodct it is typically to 'flip' the second vector and then dot it with the padded vector. 

This gives the effect of "sliding" the smaller of the two vectors across it, and taking a localized weighted sum (dot product) to store in the convolution.

This convolution has two hyperparamter 

1. Padding = how much additional zeros do we add to the vector to get
2. Stride = how far down the vector do we shift to take the next product  

Aside: This is like if we had a vector length 10 and we had a convolution of length 10, we'd only get a single number with no padding. By adding a bunch of zeros to the end of the vector, we can take local averages of the pixels toward the end of the arrays by themselves (e.g. [0, 1, 2] and [7, 8, 9])

We can use padding and stride lengths to determine the length of the output of the convolution. The three most popular modes of padding are 

1. Full = $p = m-1$ this is super largr and starts so the first element $z[0] = x[0]$. For this reason, it's rarely used.
2. Same = $p$ is selected so that the size of the output matches the input.
3. Valid = $p=0$ ensures that there is no padding at all.

Same padding is the most used in CNNs. It's typically to do a same padding CNNs layer followed by pooling or further convultional layers with higher stride lengths to decrease size for example: https://arxiv.org/abs/1412.6806

To determine the size of the convolutional output layer, we need to understand how many times we shift the filter along the input vector. The fomula is given by 

$$ o = \lfloor\frac{n + 2p -m}{s} \rfloor + 1 $$

where 
* n = input size 
* p = padding size 
* m = filter size 
* s = stride length

A naive implementation is given below: 

In [21]:
import numpy as np


def conv1d(x, y, p=0, s=1):
    y_rot = np.array(y[::-1])
    x_pad = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_pad = np.concatenate([zero_pad, x_pad, zero_pad])
    res = []
    for i in range(0, int((len(x_pad) - len(y_rot)) + 1), s):
        # range(0, int((len(x_pad) - len(w_rot))) + 1, s)
        res.append(np.sum(x_pad[i : i + y_rot.shape[0]] * y_rot))

    return np.array(res)

In [22]:
x = [1, 3, 2, 4, 5, 6, 1, 3]
y = [1, 0, 3, 1, 2]

print("Conv1D Implementation:", conv1d(x, y, p=2, s=1))

Conv1D Implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [23]:
print("Numpy results:", np.convolve(x, y, mode="same"))

Numpy results: [ 5 14 16 26 24 34 19 22]


### Performing 2D Convolutions

For a two dimensional convoultion we have 

$$Z = X * Y \rightarrow Z[i,j] = \sum_{k_1=-\infty}^{\infty}\sum_{k_2=-\infty}^{\infty}X[i-k_1, j-k_2]Y[k_1, k_2] $$

This operation has the effect of passing a filter matrix $Y$ over the matrix $X$ and creating small localized weighted means. This obviously has clear applications in image feature extraction where we want to locally pool pixels in an image. 

In [37]:
import numpy as np
import scipy.signal


def conv2d(X, Y, p=(0, 0), s=(1, 1)):
    Y_rot = np.array(Y)[::-1, ::-1]
    X_orig = np.array(X)

    n1 = X_orig.shape[0] + 2 * p[0]
    n2 = X_orig.shape[1] + 2 * p[1]

    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0] : p[0] + X_orig.shape[0], p[1] : p[1] + X_orig.shape[1]] = X_orig

    res = []
    for i in range(0, int((X_padded.shape[0] - Y_rot.shape[0]) / s[0]) + 1, s[0]):
        res.append([])
        for j in range(0, int((X_padded.shape[1] - Y_rot.shape[1]) / s[1]) + 1, s[1]):
            X_sub = X_padded[i : i + Y_rot.shape[0], j : j + Y_rot.shape[1]]
            res[-1].append(np.sum(X_sub * Y_rot))

    return np.array(res)

In [38]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
Y = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

In [39]:
x = conv2d(X, Y, p=(1, 1), s=(1, 1))
print("Conv2d Implentation:\n", x)

Conv2d Implentation:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [41]:
print("SciPy Implementation:\n", scipy.signal.convolve2d(X, Y, mode="same"))

SciPy Implementation:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


A few notes on scaling 

* You can do these way faster by using fourier transform tricks 
* It typical to have a kernel that is much smaller than the input image 
    * Typical to have (1, 1), (3, 3) and (5,5) kernels 

### Subsampling Layers

There are two types of pooling layers in CNNs which decrease the size of the feature set and reduces the dependence on one pixel in the image encouraging generalizability. 

1. max pooling = take the max in pixel neighborhood 
2. mean pooling = take the mean in pixel neighborhood 

And the neighborhod size (or pooling size) determines the size of the neighborhood. 

Max pooling is great for local heterogenity and robust to noise in the input data. 
Mean pooling 

These can be used to reduce feature size OR another convolution layer with higher stride lengths. 

To understand the differences between these two aproaches see here: https://arxiv.org/abs/1412.6806

### Implementing a CNN

They key difference between tabular ANNs and CNNs is how we process the inputs: 

* ANN: $z_1 = Wx + b$ where $x$ is a vectorized input of pixels
* CNN: $z_1 = W*X + b$ where $X$ is a matrix of pixels passed via a convolution

Since pictures could be represented in multiple colors we need to understand how to process color channels. 

Each picture will be represented by a $n_1 \times n_2 \times c$ tensor where $c$ is the number of color channels.

In [44]:
# reading in images
import torch
from torchvision.io import read_image

img = read_image("./figures/other/cat.png")

print("Image shape:", img.shape)
print("Number of channels:", img.shape[0])
print("Image data type:", img.dtype)

Image shape: torch.Size([4, 360, 360])
Number of channels: 4
Image data type: torch.uint8


In [45]:
print(img[:, 100:102, 100:102])


tensor([[[140, 132],
         [129, 135]],

        [[140, 132],
         [129, 135]],

        [[140, 132],
         [129, 135]],

        [[  0,   0],
         [  0,   0]]], dtype=torch.uint8)


Given this how can we incorperate each channel into our convolution? We do it for each channel then add them together.

Suppose each channel has it owns kernel matrix $W[::c]$ then the pre-activation inputs are given by 

$$Z^{(conv)} = \sum_{c=1}^CW[:,:,c] * X[:, :, c] $$ 
$$Z = Z^{(conv)} + b$$
$$A = \sigma(Z)$$ 

Where this last matrix $A$ is called our feature map. 


It is typical for a CNN layer to have more than one feature map which is specificed by passing different kernels over each channel. That is: 

$$Z^{(conv)}[:,:,k] = \sum_{c=1}^CW[:,:,c, k] * X[:, :, c]$$ 
$$Z[:,:,k] = Z^{(conv)}[:,:,k] + b[k]$$
$$A[:,:,k] = \sigma(Z[:,:,k)$$

which supplies us with $K$ different feature maps which can be used in subsequent layers for learning. 



So the steps are as follows: 

1. For each kernel $K$, convolve over each channel $C$ and add them up. 
2. Do this for each kernel which provides $K$ new elements to work with.
3. Perform max pooling with also results in $K$ elements but of possible smaller dimesion

Aside: This was not super well explained. The point is that you can create individual feature maps by passing different kerels over each channel. Aggregating those feature maps serves as useful inputs for down the network. Morever, the network can learn what these kernels should be to maximize impact of these feature maps.

__Regularization__

These models truly have an insane number of weights so learning how to perform regularization seems really important. 

We'll disuss $\ell_2$ regularization by regularizating the parameters in each layer and _dropout_ which randomly drops nodes from the network during training to ensure no one section of the network becomes too important and removes ReLu dead neurons. 

In [46]:
# ell_2 regularization

import torch.nn as nn

loss_func = nn.BCELoss()
loss = loss_func(torch.tensor([0.9]), torch.tensor([1.0]))
l2_lambda = 0.001

conv_layer = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=5)
l2_penalty = l2_lambda * sum([(p**2).sum() for p in conv_layer.parameters()])
loss_with_penalty = loss + l2_penalty

linear_layer = nn.Linear(10, 16)
l2_penalty = l2_lambda * sum([(p**2).sum() for p in linear_layer.parameters()])
loss_with_penalty = loss + l2_penalty

While we can use this [droput](https://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf) remains the most important regularization method. 

During training, we will randomly drop neurons with a certain probability $p$ (commonly set to $p=0.5$). This forces the network to learn _redudant information in the training data_ increasing the model's robustness.  

Implicitly this technique is performing ensemble learning as it learns different parts of feature-label relationship during each epoch. 

Aside: There is a very out of place discussion on the difference between categorial and binary classification and the differ ways to implement in in PyTorch which we'll now code up.

In [48]:
# Binary Cross Entropy
logits = torch.tensor([0.8])
probs = torch.sigmoid(logits)
target = torch.tensor([1.0])
bce_loss_fn = nn.BCELoss()
bce_logits_loss_fn = nn.BCEWithLogitsLoss()
print(f"BCE (with Probs): {bce_loss_fn(probs, target):.4f}")
print(f"BCE (with Logits): {bce_logits_loss_fn(logits, target):.4f}")

BCE (with Probs): 0.3711
BCE (with Logits): 0.3711


In [53]:
# Categorical Cross Entropy
logits = torch.tensor([[1.5, 0.8, 2.1]])
probs = torch.softmax(logits, dim=1)
target = torch.tensor([2])
cce_loss_fn = nn.NLLLoss()
cce_logits_loss_fn = nn.CrossEntropyLoss()
print(f"CCE (with Probs): {cce_loss_fn(torch.log(probs), target):.4f}")
print(f"CCE (with Logits): {cce_logits_loss_fn(logits, target):.4f}")

CCE (with Probs): 0.5996
CCE (with Logits): 0.5996
